In [1]:
import pandas as pd
import io
import random
import time
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import re
import string
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from rake_nltk import Rake
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import warnings
from sklearn.feature_extraction.text import HashingVectorizer
warnings.filterwarnings('ignore')

#nltk.download('popular', quiet=True) # for downloading packages
#!pip3 install rake-nltk

def recommend_movie_choices():
    print("\nROBO: So you want to see a new movie now right?")
    df_movies = pd.read_csv("IMDb movies.csv")
    df_ratings = pd.read_csv("IMDb ratings.csv")

    # Taking a look at all the columns in the dataframe(s)
    #print("Columns for Movie Details: \n",df_movies.columns)
    #print("Columns for Movie Rating statistics: \n", df_ratings.columns)

    # Taking a look at the Movie Details Dataframe
    #df_movies.head()

    # Taking a look at the Movie Rating Statistics Dataframe
    #df_ratings.head()

    # Removing all the unwanted columns from the two Dataframes
    df_movies = df_movies[['imdb_title_id','title', 'duration', 'year', 'genre', 'language', 'actors', 'director','description']]
    df_ratings = df_ratings[['imdb_title_id', 'mean_vote', 'weighted_average_vote','median_vote', 'total_votes']]

    #Again Taking a look at all the columns in the dataframe(s) after dropping unwanted columns
    #print("Columns for Movie Details: \n",df_movies.columns)
    #print("Columns for Movie Rating statistics: \n", df_ratings.columns)

    # Merging the two dataframes and dropping all the nan values
    df = pd.merge(df_movies, df_ratings, on='imdb_title_id')
    #print("Shape, Before dropping Nan Values: ",df.shape)
    df.dropna(inplace = True)
    dfm = df.copy()
    #print("Shape, After dropping Nan Values: ",df.shape)

    df2 = df[df['language'].str.contains(r'English')]
    #print(df2.shape)
    df2 = df2[(df2['mean_vote'] >= 6) & (df['total_votes'] >= 1000)] # Take all English Movies with Rating greater than 6
    #print(df2.shape)
    df2 = df2[df2['year'] >= 1995]
    #print(df2.shape)
    df2[df2['title'].str.contains('123')]

    df3 = df[df['language'].str.contains(r'Tamil|Kannada|Telugu|Hindi|Malayalam')]
    #df3.shape
    df3 = df3[(df3['mean_vote'] >= 5) & (df3['total_votes'] >= 500)]
    df3[df3['title'].str.contains('Student')]
    #df3.shape


    df = pd.concat([df2,df3])
    df = df.apply(lambda x: x.str.lower() if(x.dtype == 'O') else x)
    df = df.drop_duplicates(subset=['title','year'], keep = False)

    dfm = dfm[(dfm['mean_vote'] >= 5) & (dfm['total_votes'] >= 500)]
    dfm = dfm.sort_values(by=['mean_vote'],ascending = False)
    dfm = dfm.apply(lambda x: x.str.lower() if(x.dtype == 'O') else x)
    df.shape
    time.sleep(3.4)
    print("\n ROBO: So to make life easy fo you I have already made a list of some of the popular movies since the 2000s in English, Hindi, Tamil and some other Indian regional languages.")
    print("ROBO: ", end='')
    time.sleep(2.1)
    print("So do you have any preferences ?")
    print("ROBO: Want to search from your own list?")

    ch = input().lower()
    if ('yes' in ch) | ('yea' in ch) | ('ya' in ch) | ('ye' in ch):
        # Accepting user input to identify similar movies of their interest
        gen = input("ROBO: Cool!!\nEnter Preferred genre(s) (if more than one please use a comma)(Type No if not): ").lower()
        df2 = dfm.copy()
        if gen != 'no':
            df2 = dfm[dfm['genre'].str.contains(gen)]
            df2 = df2.drop_duplicates(subset=['title'], keep = False)

        lang = input("ROBO: Any Preferred Language(s) (if more than one please use a comma)(Type No if not): ").lower()
        df3 = df2.copy()
        if lang != 'no' :
            df3 = df2[df2['language'].str.contains(lang)]
            df3 = df3.drop_duplicates(subset = ['title'],keep=False)
    else:
        print("\nROBO: I see you believe in me :)")
        df3 = df
    df3 = df3.sort_values(by=['mean_vote'],ascending = False)
    if df3.shape[0] > 10000:
        df3 = df3[:10000]
    if df3.shape[0] == 0:
        print("ROBO: I'm sorry but you have not selected any movies. Please try again")
    df3.shape

    df3.reset_index(drop=True,inplace=True)

    df3['Key_words'] = ''
    r = Rake()
    for index, row in df3.iterrows():
        r.extract_keywords_from_text(row['description'])
        key_words_dict_scores = r.get_word_degrees()
        row['Key_words'] = list(key_words_dict_scores.keys())
        df3['Key_words'][index] = row['Key_words']

    df3['genre'] = df3['genre'].map(lambda x: x.split(','))
    for index, row in df3.iterrows():
        row['genre'] = [x.lower().replace(' ','') for x in row['genre']]

    df3['Bag_of_words'] = ''
    columns = ['Key_words', 'genre']
    for index, row in df3.iterrows():
        words = ''
        for col in columns:
            words += ' '.join(row[col]) + ' '
        row['Bag_of_words'] = words
        df3['Bag_of_words'][index] = words
        dfn = df3[['title','Bag_of_words']]

    count = CountVectorizer()
    count_matrix = count.fit_transform(dfn['Bag_of_words'])
    from sklearn.metrics.pairwise import cosine_similarity

    def cosine_similarity_n_space(m1, m2, batch_size=10000):
        assert m1.shape[1] == m2.shape[1]
        ret = np.ndarray((m1.shape[0], m2.shape[0]))
        for row_i in range(0, int(m1.shape[0] / batch_size) + 1):
            start = row_i * batch_size
            end = min([(row_i + 1) * batch_size, m1.shape[0]])
            if end <= start:
                break # cause I'm too lazy to elegantly handle edge cases
            rows = m1[start: end]
            sim = cosine_similarity(rows, m2) # rows is O(1) size
            ret[start: end] = sim
        return ret

    csmain = cosine_similarity_n_space(count_matrix, count_matrix)
    dfn = df3[['title','Bag_of_words']]
    count = CountVectorizer()
    count_matrix = count.fit_transform(dfn['Bag_of_words'])
    csmain = cosine_similarity_n_space(count_matrix, count_matrix)
    indices = pd.Series(dfn['title'])
    def recommend(title, num=10, cosine_sim = csmain):
        recommended_movies = []
        try:
            idx = indices[indices == title].index[0]
            score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
            top_10_indices = list(score_series.iloc[1:num+1].index)

            for i in top_10_indices:
                recommended_movies.append(list(dfn['title'])[i])
            print("\nGreat Choice. Here is the list of similar movies:")
            for i in recommended_movies:
                print(i.title())
            return False
        except:
            print("ROBO: I'm sorry but I could not find such a movie in our Database.")
            print("ROBO: I'd recommend you to check the spelling of the movie you entered.")
            print("ROBO: Also make sure it belongs to the same genre and language you had entered before.")
            return True

    ans = True
    while (ans):
        print("ROBO: Please refer to Imdb for the exact movie Name.")
        user_res = input("Enter the movie which you have in mind.").lower()
        num = int(input("How many such similar movies do you want??"))
        ans = recommend(user_res,num)

In [2]:
data = pd.read_csv("labeledTrainData.tsv",sep="\t")
data.head()
X = data.review
y = data.sentiment
#Using CountVectorizer to convert text into tokens/features
vect = CountVectorizer(stop_words='english', ngram_range = (1,1), max_df = .80, min_df = 4)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1, test_size= 0.2)
#Using training data to transform text into counts of features for each message
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_test_dtm = vect.transform(X_test)

NB = MultinomialNB()
NB.fit(X_train_dtm, y_train)
y_pred = NB.predict(X_test_dtm)
#print('\nNaive Bayes')
#print('Accuracy Score: ',metrics.accuracy_score(y_test,y_pred)*100,'%',sep='')
#print('Confusion Matrix: ',metrics.confusion_matrix(y_test,y_pred), sep = '\n')

tokens_words = vect.get_feature_names()
#print('\nAnalysis')
#print('No. of tokens: ',len(tokens_words))
counts = NB.feature_count_
df_table = {'Token':tokens_words,'Negative': counts[0,:],'Positive': counts[1,:]}
tokens = pd.DataFrame(df_table, columns= ['Token','Positive','Negative'])
positives = len(tokens[tokens['Positive']>tokens['Negative']])
#print('No. of positive tokens: ',positives)
#print('No. of negative tokens: ',len(tokens_words)-positives)
#Check positivity/negativity of specific tokens
token_search = ['horrendous']
#print('\nSearch Results for token/s:',token_search)
#print(tokens.loc[tokens['Token'].isin(token_search)])
#Analyse False Negatives (Actual: 1; Predicted: 0)(Predicted negative review for a positive review) 
#print(X_test[ y_pred < y_test ])
#Analyse False Positives (Actual: 0; Predicted: 1)(Predicted positive review for a negative review) 
#print(X_test[ y_pred > y_test ])

In [4]:
flag=True
print("ROBO: Hi there")
print("ROBO: My name is Robo. Type 'help' for guidance, 'review' to review a movie and let others know your opinion, 'recommend' so we can recommend you a movie based on your preference :)")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='recommend'):
        if(user_response!='review'):
            if(user_response=='thanks' or user_response=='thank you' ):
                flag=False
                print("ROBO: You are welcome..")
            else:
                if(greeting(user_response)!=None):
                    print("ROBO: "+greeting(user_response))
                else:
                    print("ROBO: ",end="")
                    print(response(user_response))
                    sent_tokens.remove(user_response)
        else:
            flag=False
            print("ROBO: Alright...")  
    else:
        flag=False
        print("ROBO: Alright...")    
        
if(user_response=='recommend'):
    string = input("Enter a movie based on which we could recommend one to you: ")
    print("Similar movies are")
    string = string.lower()
    #recommend(string)
    print(recommend(string))
elif(user_response=='review'):
    trainingVector = CountVectorizer(stop_words='english', ngram_range = (1,1), max_df = .80, min_df = 5)
    trainingVector.fit(X)
    X_dtm = trainingVector.transform(X)
    NB_complete = MultinomialNB()
    NB_complete.fit(X_dtm, y)
    #Input Review
    print('\nTest a custom review message')
    print('Enter review to be analysed: ', end=" ")
    test = []
    test.append(input())
    test_dtm = trainingVector.transform(test)
    predLabel = NB_complete.predict(test_dtm)
    tags = ['Negative','Positive']
    #Display Output
    print('The review is predicted:',tags[predLabel[0]])
else:
    print('Bye! Have a good day')

ROBO: Hi there
ROBO: My name is Robo. Type 'help' for guidance, 'review' to review a movie and let others know your opinion, 'recommend' so we can recommend you a movie based on your preference :)
recommend
ROBO: Alright...


KeyboardInterrupt: 